## Chatbot del Reglamento de tránsito de la Ciudad de México 

In [1]:
#Librerías 
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

In [2]:
#Iniciamos nuestro modelo
openai_api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(openai_api_key=openai_api_key)
llm.invoke("how can langsmith help with testing?")

AIMessage(content="Langsmith can help with testing in several ways:\n\n1. Automated testing: Langsmith can generate test cases automatically, saving time and effort in writing test cases manually. It can generate a wide range of test cases, including both positive and negative scenarios, to ensure comprehensive coverage of the system under test.\n\n2. Test data generation: Langsmith can generate realistic and diverse test data, which is crucial for testing the system's behavior under various conditions. It can create different combinations of inputs to uncover edge cases and potential issues.\n\n3. Test case optimization: Langsmith can analyze existing test cases and identify redundant or overlapping test cases. By removing unnecessary test cases, it helps in reducing the testing effort while maintaining the same level of coverage.\n\n4. Bug detection: Langsmith can analyze code and identify potential bugs or vulnerabilities. It can simulate different execution paths and identify areas

In [3]:
# Definimos un mensaje introductorio de nuestro Chatbot 
system_intro = "Hola. Soy una asistente de AI con un extenso conocimiento sobre las regulaciones de tránsito de la CDMX."


In [4]:
# Definimos la historia del chat
chat_history = [
    HumanMessage(content="Puede un vehiculo circular con poliza de seguro vencida?"),
    AIMessage(content="No!")
]

In [5]:
# Input del usuario 
user_input = "Dime por qué?"

In [6]:
# Función para procesar el input del usuario
def process_user_input(user_input):
    # Remove leading and trailing whitespaces
    user_input = user_input.strip()

    # Check if the user input is empty
    if not user_input:
        print("Error: Please provide a valid input.")
        return None

    # Additional input processing logic if needed...

    return user_input

In [19]:
def invoke_chatbot(input_data, chat_history=None):
    try:
        # Get the system introduction
        system_intro = "You are an AI assistant with extensive knowledge of traffic regulations in Mexico City."

        # Generate the full input using the system introduction, chat history, and user input
        full_input = generate_full_input(input_data, system_intro, chat_history)

        # Prepare the input data with optional chat history
        input_payload = {"input": full_input}

        # Invoke the chatbot
        response = retrieval_chain.invoke(input_payload)

        # Extract and print the answer
        answer = response.get("answer", "Sorry, I couldn't generate a valid response.")
        print("Chatbot Response:", answer)

        return answer

    except Exception as e:
        print(f"An error occurred while invoking the chatbot: {str(e)}")
        # Handle the error gracefully, e.g., log the error, inform the user, or return an error message
        return "Sorry, an error occurred while processing your request."

In [20]:
def process_and_invoke(user_input, chat_history=None):
    processed_input = process_user_input(user_input)

    # Check if the input is valid before proceeding
    if processed_input is not None:
        # Generate the full input including system introduction and chat history
        full_input = generate_full_input(processed_input, system_intro, chat_history)
        invoke_chatbot(full_input, chat_history)

# Example usage
#process_and_invoke(user_input, chat_history)

## Template introductorio

In [21]:
# Template de la introducción y el input del usuario
custom_prompt = ChatPromptTemplate.from_messages([
    ("system", system_intro),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])


- Carga de los datos 

In [22]:
# Carga del PDF
loader = PyPDFLoader("raw_data/Reglamento_CDMX.pdf")
pages = loader.load_and_split()

In [23]:
# Creación de vectores y embeddings
embeddings = OpenAIEmbeddings()
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(pages)
vector = FAISS.from_documents(documents, embeddings)

In [24]:
# Creación de la cadena de documentos 
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:
<context>
{context}
</context>
Question: {input}""")
document_chain = create_stuff_documents_chain(llm, prompt)

In [25]:
# Creación del Retrieval Chain
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [26]:
# Example usage of retrieval chain
response = retrieval_chain.invoke({"input": "que requisito necesita un menor de edad para circular un vehiculo?"})
print(response["answer"])

Según el contexto proporcionado, un menor de edad necesita ser capaz de sujetarse por sí mismo a la motocicleta y colocar adecuada y firmemente los pies en los estribos o posa pies, a menos que cuente con aditamentos especialmente diseñados para su seguridad.


## Creación completa del Input final 

In [27]:
def generate_full_input(user_input, system_intro, chat_history=None):
    # Combine the system introduction, chat history, and user input
    full_input = f"{system_intro}\n\n"
    if chat_history:
        for message in chat_history:
            full_input += f"{message.content}\n"
    full_input += f"User: {user_input}"
    return full_input

In [28]:
# Example usage of retrieval chain
response = retrieval_chain.invoke({"input": generate_full_input("que requisito necesita un menor de edad para circular un vehiculo?", system_intro)})
print(response["answer"])

Según el contexto proporcionado, un menor de edad necesita poder sujetarse por sí mismo a la motocicleta y poder colocar adecuada y firmemente los pies en los estribos o posa pies, a menos que cuente con aditamentos especialmente diseñados para su seguridad.


In [29]:
process_and_invoke("que requisito necesita un menor de edad para circular un vehiculo?", chat_history)

Chatbot Response: En el contexto proporcionado no se menciona ningún requisito específico para que un menor de edad pueda circular un vehículo. Sin embargo, generalmente en México se requiere que los menores de edad cuenten con una licencia de conducir o permiso especial, dependiendo de la edad y las regulaciones específicas de cada estado. Es importante consultar la legislación local y cumplir con todos los requisitos legales antes de permitir que un menor de edad conduzca un vehículo.


In [30]:
process_and_invoke("se puede conducir un vehiculo sin poliza de seguro vigente?", chat_history)

Chatbot Response: No, no se puede conducir un vehículo sin una póliza de seguro vigente. Es obligatorio tener una póliza de seguro de responsabilidad civil vigente para circular en la Ciudad de México.


In [31]:
user_input = "Dime por qué?"

# Process and invoke the chatbot
invoke_chatbot(user_input, chat_history)

Chatbot Response: Un vehículo no puede circular con una póliza de seguro vencida porque es una obligación legal que los vehículos cuenten con una póliza de seguro vigente que ampare la responsabilidad civil por daños a terceros en su persona y en su patrimonio. Circular sin un seguro válido es una infracción y puede resultar en multas y sanciones según las regulaciones de tránsito.


'Un vehículo no puede circular con una póliza de seguro vencida porque es una obligación legal que los vehículos cuenten con una póliza de seguro vigente que ampare la responsabilidad civil por daños a terceros en su persona y en su patrimonio. Circular sin un seguro válido es una infracción y puede resultar en multas y sanciones según las regulaciones de tránsito.'